In [ ]:
!apt install -y python-opengl ffmpeg xvfb
!pip3 install pyvirtualdisplay stable-baselines3[extra] gymnasium huggingface_sb3 huggingface_hub panda_gym moviepy

In [ ]:
from pyvirtualdisplay import Display

import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
env_id = "PandaReachDense-v3"

env = gym.make(env_id)

s_size = env.observation_space.shape
a_size = env.action_space

In [ ]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [ ]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

In [ ]:
model.learn(1e6)

In [ ]:
model.save("a2c_panda_reach_dense_v3")
env.save("vec_normalize.pkl")

In [ ]:
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"

eval_env.training = False
eval_env.norm_reward = False

model = A2C.load("a2c_panda_reach_dense_v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
notebook_login()

In [ ]:
model_name = "a2c_panda_reach_dense_v3"
package_to_hub(
    model=model,
    model_name=model_name,
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"jaymanvirk/{model_name}",
    commit_message="initial commit",
)